# Pandas Lab — FAANG-Level Hands-On

**Goal:** Build strong intuition for Pandas transformations used in ML pipelines (joins, groupby, leakage-safe features).

**Outcome:** You can write correct, scalable Pandas code and explain common gotchas (join explosion, leakage, apply misuse).


In [1]:
import pandas as pd
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')


## Section 0 — Synthetic Dataset (Interview-Friendly)
We’ll use synthetic tables to mirror typical ML feature engineering tasks.

In [2]:
rng = np.random.default_rng(0)

users = pd.DataFrame({
    'user_id': [101, 102, 103, 104],
    'country': ['IN', 'US', 'IN', 'CA'],
    'signup_ts': pd.to_datetime(['2025-01-01', '2025-01-03', '2025-01-04', '2025-01-10'])
})

events = pd.DataFrame({
    'user_id': rng.choice(users['user_id'], size=30, replace=True),
    'event_ts': pd.to_datetime('2025-01-01') + pd.to_timedelta(rng.integers(0, 20, size=30), unit='D'),
    'event': rng.choice(['view', 'add_to_cart', 'purchase'], size=30, replace=True, p=[0.7, 0.2, 0.1]),
    'amount': rng.choice([0, 0, 0, 19.99, 49.99, 99.99], size=30, replace=True),
})

# Ensure amount > 0 only for purchases (simple realism)
events.loc[events['event'] != 'purchase', 'amount'] = 0

users, events.head()


(   user_id country  signup_ts
 0      101      IN 2025-01-01
 1      102      US 2025-01-03
 2      103      IN 2025-01-04
 3      104      CA 2025-01-10,
    user_id   event_ts        event  amount
 0      104 2025-01-17         view     0.0
 1      103 2025-01-04         view     0.0
 2      103 2025-01-02         view     0.0
 3      102 2025-01-18  add_to_cart     0.0
 4      102 2025-01-01         view     0.0)

## Section 1 — DataFrame Fundamentals

### Task 1.1: Basic filtering + new columns

Create:
- `purchases`: rows where `event == 'purchase'`
- add column `event_day` = date (no time)

# HINT:
- Use boolean indexing
- Use `.dt.floor('D')` or `.dt.date` (but prefer datetime64)

**Explain:** Why does vectorized `.dt` beat per-row parsing?


In [22]:
# TODO
purchases = events[events['event']=='purchase']
purchases = purchases.copy()

print (purchases)

purchases['event_day'] = purchases['event_ts'].dt.floor('D')

check('purchases_only', set(purchases['event'].unique()) <= {'purchase'})
check('event_day_dtype', str(purchases['event_day'].dtype).startswith('datetime'))


   user_id   event_ts     event  amount
8      101 2025-01-10  purchase   99.99
OK: purchases_only
OK: event_day_dtype


## Section 2 — GroupBy (Core for Features)

### Task 2.1: Per-user aggregates
Compute per-user features:
- `n_events`
- `n_purchases`
- `total_revenue` (sum of amount)

# HINT:
- Use `groupby('user_id').agg(...)`
- For `n_purchases`, use conditional aggregation

**FAANG gotcha:** Counting purchases using `amount>0` vs `event=='purchase'` can diverge if data is messy.


In [10]:
# TODO
user_features = (
    events.assign(is_purchase= (events['event']=='purchase').astype(int)).groupby('user_id', as_index=True).agg(
        n_events = ('event', 'size'),
        n_purchases = ('is_purchase','sum'),
        total_revenue = ('amount','sum')
    )
)

user_features = user_features.reset_index()
print(user_features)
check('has_user_id', 'user_id' in user_features.columns)
check('has_total_revenue', 'total_revenue' in user_features.columns)


   user_id  n_events  n_purchases  total_revenue
0      101         6            1          99.99
1      102         4            0           0.00
2      103        12            0           0.00
3      104         8            0           0.00
OK: has_user_id
OK: has_total_revenue


### Task 2.2: GroupBy transform (row-level feature)

Add a column to `events` called `user_event_count` = number of events for that user.

# HINT:
- Use `groupby(...).transform('count')`

**Explain:** Why is `transform` different from `agg`?


In [11]:
# TODO
events2 = events.copy()
events2['user_event_count'] = events2.groupby('user_id')['event'].transform('size')
#events2['user_event_count'] = events2.groupby('user_id').transform('size')
#events2['user_event_count'] = events2.groupby('user_id').transform('count')

print (events2)

# Cannot use agg here because we want to add a new column - with the same number of rows that the original data frame has. agg returns a collapsed dataframe

check('user_event_count_nonnull', events2['user_event_count'].notna().all())
check('counts_positive', (events2['user_event_count'] > 0).all())


    user_id   event_ts        event  amount  user_event_count
0       104 2025-01-17         view    0.00                 8
1       103 2025-01-04         view    0.00                12
2       103 2025-01-02         view    0.00                12
3       102 2025-01-18  add_to_cart    0.00                 4
4       102 2025-01-01         view    0.00                 4
5       101 2025-01-11         view    0.00                 6
6       101 2025-01-02         view    0.00                 6
7       101 2025-01-06  add_to_cart    0.00                 6
8       101 2025-01-10     purchase   99.99                 6
9       104 2025-01-09         view    0.00                 8
10      103 2025-01-09         view    0.00                12
11      104 2025-01-01         view    0.00                 8
12      103 2025-01-01         view    0.00                12
13      103 2025-01-03         view    0.00                12
14      104 2025-01-01         view    0.00                 8
15      

## Section 3 — Joins & Merge (Feature Table Building)

### Task 3.1: Join user features back to users

Create `user_table` by joining `users` with `user_features` on `user_id`.

# HINT:
- Use `merge(..., how='left')`
- Fill missing aggregates with 0

**FAANG gotcha:** If you accidentally do a many-to-many join, row count explodes. Always validate row counts.


In [13]:
# TODO
user_table = users.merge(user_features, on='user_id', how='left')

# Fill NaNs for users with no events
for col in ['n_events', 'n_purchases', 'total_revenue']:
    if col in user_table.columns:
        user_table[col] = user_table[col].fillna(0)

check('rowcount_preserved', len(user_table) == len(users))
print(user_table)


OK: rowcount_preserved
   user_id country  signup_ts  n_events  n_purchases  total_revenue
0      101      IN 2025-01-01         6            1          99.99
1      102      US 2025-01-03         4            0           0.00
2      103      IN 2025-01-04        12            0           0.00
3      104      CA 2025-01-10         8            0           0.00


### Task 3.2: Join explosion debugging (mini)

Construct a tiny example where a join becomes many-to-many and explodes rows. Then fix it.

# HINT:
- Create `left` with duplicate keys
- Create `right` with duplicate keys
- Merge and observe rowcount

**Explain:** What join cardinality do you expect (1:1, 1:n, n:1, n:n)?


In [14]:
# TODO
left = pd.DataFrame({'k': [1, 1, 2], 'l': ['a', 'b', 'c']})
right = pd.DataFrame({'k': [1, 1, 2], 'r': ['x', 'y', 'z']})
exploded = left.merge(right, on='k', how='inner')
print('left rows', len(left), 'right rows', len(right), 'merged rows', len(exploded))



# TODO: Fix (example: deduplicate right to 1 row per k)
right_dedup = right.drop_duplicates('k',keep='first')
fixed = left.merge(right_dedup, on='k', how='inner')
print('fixed rows', len(fixed))

left rows 3 right rows 3 merged rows 5
fixed rows 3


## Section 4 — Time-based Features & Leakage (FAANG System Thinking)

### Task 4.1: Leakage-safe feature

For each user and each event row, compute `purchases_before` = number of purchases strictly BEFORE that `event_ts`.

Constraints:
- No Python loops over rows
- Must be time-aware

# HINT:
- Sort by user_id, event_ts
- Create boolean `is_purchase`
- Use groupby + cumsum, then shift

**FAANG gotcha:** If you include the current row in the count, you leak label info for purchase events.


In [15]:
# TODO
events3 = events.copy().sort_values(['user_id', 'event_ts']).reset_index(drop=True)
events3['is_purchase'] = (events3['event'] == 'purchase').astype(int)

# purchases up to and including current row
events3['purchases_cum'] = events3.groupby('user_id')['is_purchase'].cumsum()

# TODO: leakage-safe: purchases strictly before current row
events3['purchases_before'] = events3.groupby('user_id')['purchases_cum'].shift(1).fillna(0).astype(int)

check('nonnegative', (events3['purchases_before'] >= 0).all())
print(events3[['user_id','event_ts','event','purchases_before']].head(10))

OK: nonnegative
   user_id   event_ts        event  purchases_before
0      101 2025-01-02         view                 0
1      101 2025-01-06  add_to_cart                 0
2      101 2025-01-10     purchase                 0
3      101 2025-01-11         view                 1
4      101 2025-01-14  add_to_cart                 1
5      101 2025-01-20         view                 1
6      102 2025-01-01         view                 0
7      102 2025-01-16  add_to_cart                 0
8      102 2025-01-17         view                 0
9      102 2025-01-18  add_to_cart                 0



## Section 5 — Apply vs Vectorization

### Task 5.1: Remove slow apply

Create a `country_is_in` boolean column on `users`.
- First: do it with `.apply(...)` (slow style)
- Then: replace it with vectorized code

**Explain:** Why is `.apply` often slower in Pandas?


In [33]:
import time

u = users.copy()

# TODO: slow style (keep for comparison)

t1 = time.time()
u['country_is_in_apply'] = u['country'].apply(lambda x: x == 'IN')
t2 = time.time()
print (u)

# TODO: vectorized style
t3= time.time()
u['country_is_in'] = u['country'] == 'IN'
#u['country'][u['country'] == 'IN'] = 'MA'
#u.loc[u['country']=='IN', 'country'] = "LA"
t4 =time.time()

print ("apply speed      :", t2-t1)
print ("vectorized speed :", t4-t3)

#check('same_result', (u['country_is_in_apply'] == u['country_is_in']).all())
print (u)

   user_id country  signup_ts  country_is_in_apply
0      101      IN 2025-01-01                 True
1      102      US 2025-01-03                False
2      103      IN 2025-01-04                 True
3      104      CA 2025-01-10                False
apply speed      : 0.0010058879852294922
vectorized speed : 0.0010173320770263672
   user_id country  signup_ts  country_is_in_apply  country_is_in
0      101      IN 2025-01-01                 True           True
1      102      US 2025-01-03                False          False
2      103      IN 2025-01-04                 True           True
3      104      CA 2025-01-10                False          False


---
## Submission Checklist
- All TODOs completed
- All checks pass
- Explain prompts answered
